In [47]:
import sys
try:
    import distutils
except ModuleNotFoundError:
    import setuptools._distutils as distutils
    sys.modules["distutils"] = distutils

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count
from pyspark.sql.functions import explode, split


spark = SparkSession.builder \
    .appName("MoviesRatings") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [49]:
movies = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies.show(10)
ratings.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|

In [50]:
movies.printSchema()
ratings.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [51]:
print(f"Nombre total de films : {movies.count()}")
print(f"Nombre total de notes : {ratings.count()}")

Nombre total de films : 9742
Nombre total de notes : 100836


In [52]:
print(f"Nombre d'utilisateurs uniques : {ratings.select('userId').distinct().count()}")
print(f"Nombre de films notés : {ratings.select('movieId').distinct().count()}")

Nombre d'utilisateurs uniques : 610
Nombre de films notés : 9724


In [53]:
ratings.describe("rating").show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100836|
|   mean| 3.501556983616962|
| stddev|1.0425292390606342|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [54]:
from pyspark.sql.functions import col
ratings.groupBy("rating").count().orderBy(col("rating")).show()

+------+-----+
|rating|count|
+------+-----+
|   0.5| 1370|
|   1.0| 2811|
|   1.5| 1791|
|   2.0| 7551|
|   2.5| 5550|
|   3.0|20047|
|   3.5|13136|
|   4.0|26818|
|   4.5| 8551|
|   5.0|13211|
+------+-----+



In [55]:
best_movies = ratings.groupBy("movieId") \
    .agg(avg("rating").alias("avg_rating"), count("rating").alias("count_rating")) \
    .filter("count_rating >= 10") \
    .orderBy(col("avg_rating").desc())
best_movies.show(10)

+-------+-----------------+------------+
|movieId|       avg_rating|count_rating|
+-------+-----------------+------------+
|   1041|4.590909090909091|          11|
|   3451|4.545454545454546|          11|
|   1178|4.541666666666667|          12|
|   1104|            4.475|          20|
|   2360|4.458333333333333|          12|
|   1217|4.433333333333334|          15|
|    318|4.429022082018927|         317|
|    951|4.392857142857143|          14|
|   1927|             4.35|          10|
|    922|4.333333333333333|          27|
+-------+-----------------+------------+
only showing top 10 rows



In [56]:
avg_ratings_per_movie = ratings.groupBy("movieId").count().agg({"count": "avg"})
avg_ratings_per_movie.show()

+------------------+
|        avg(count)|
+------------------+
|10.369806663924312|
+------------------+



In [57]:
movies_genres = movies.withColumn("genre", explode(split(col("genres"), "\\|")))
movies_genres.groupBy("genre").count().orderBy(col("count").desc()).show()


+------------------+-----+
|             genre|count|
+------------------+-----+
|             Drama| 4361|
|            Comedy| 3756|
|          Thriller| 1894|
|            Action| 1828|
|           Romance| 1596|
|         Adventure| 1263|
|             Crime| 1199|
|            Sci-Fi|  980|
|            Horror|  978|
|           Fantasy|  779|
|          Children|  664|
|         Animation|  611|
|           Mystery|  573|
|       Documentary|  440|
|               War|  382|
|           Musical|  334|
|           Western|  167|
|              IMAX|  158|
|         Film-Noir|   87|
|(no genres listed)|   34|
+------------------+-----+

